# Let's talk about machine learning


The phrase "machine learning" refers to any one of a number of methods for approximating a solution to a problem for which we don't have an analytical solution (an algorithmic solution) through examining data. The basic taxonomy of machine learning approaches is depicted below (from a SAS blog, see embedded link):

![ML algorithms](https://blogs.sas.com/content/subconsciousmusings/files/2017/04/machine-learning-cheet-sheet-2.png)

However, note that one can "mimic" supervised learning with minimal labeling via self-supervised learning or semi-supervised learning. Also note that this diagram does not include a third major class of ML algorithm, reinforcement learning.

When we talk about machine learning, we talk about:
* *fitting* (or *training*) a *prediction function*, or *model*, to
* *training* data, experimenting with various
* *hyperparameters* related to the *model architecture* using held-out
* *development* data, so that the resulting model generalizes well, making good *predictions* on held-out
* *test* (or *evaluation*) data

The goal of unsupervised learning is to uncover latent structure or patterns in the data. The goal of supervised learning is to learn to match the labels (or answers, or ground truth, or dependent variable) in the data.

We evaluate models / prediction functions using any number of metrics, but a commonly used one for supervised machine learning is:
* accuracy - what percent of the data points were classified correctly?

Of course, accuracy is just one number. To get a clearer understanding, we can construct a
* confusion matrix,
which has the classes (the labels) along rows and columns, and in each cell indicates the number of data points classified as *row* that are truly in class *column*.

# K-nearest neighbors

K-nearest neighbors (KNNs) is a very very simple supervised ML algorithm.
* fit - it just stores all the training data!
* predict - it finds the data points in the training data that are closest to the data point to be classified, and takes a majority vote of their labels

# Questions:
What are some of the hyperparameters related to kNN?

How would you pick good values for them?

kNN is expensive in terms of computational cost and memory. You can address each of these costs using methods *we already know*.
* to make it faster, what can you do?
* To make it less memory intensive, what can you do?